# Colour Correction using Splines

### Define constants

In [2]:
%load_ext autoreload
%autoreload 2
import colour
from data import load_dataset_sfu, load_dataset_csv, load_cmfs, load_camera, load_insitu, msds_to_rgb, msds_to_xyz, load_dataset_skin
import numpy as np

RANDOM_STATE = 0
np.random.seed(RANDOM_STATE)
    
SFU_FILE_PATH = 'data/reflect_db.reflect'
CAVE_FOSTER2004_PATH = 'data/cave_foster2002.csv'
INSITU_PATH = "data/insitu_dataset.csv"
CAVE_PATH = 'data/cave.csv'
FOSTER_50_PATH = 'data/foster50.csv'
CAMERA = 'sigma'

In [3]:

# sfu_dataset = load_dataset_sfu(SFU_FILE_PATH)
cave_foster2004_dataset = load_dataset_csv(CAVE_FOSTER2004_PATH)
foster_50_dataset = load_dataset_csv(FOSTER_50_PATH)
insitu = load_insitu(INSITU_PATH)
# cave = load_dataset_csv(CAVE_PATH)
# skin = load_dataset_skin()


In [4]:
from plotting import plot_chromaticity_diagram
TRAIN = cave_foster2004_dataset
TEST = foster_50_dataset
VALIDATION = insitu

### Computing Observer Responses
We can easily change the order of test and train sets here

In [5]:
from plotting import plot_chromaticity_diagram
from colour.characterisation import training_data_sds_to_XYZ

cmfs, illuminant = load_cmfs()
response_trainset_xyz = msds_to_xyz(TRAIN, cmfs, illuminant)
response_testset_xyz = msds_to_xyz(TEST, cmfs, illuminant)
response_validation_xyz = msds_to_xyz(VALIDATION, cmfs, illuminant)


[ 0.94940092  1.          1.08709122]
[ 0.94940092  1.          1.08709122]
[ 0.94940092  1.          1.08709122]


### Computing Camera Responses

In [37]:
from colour.characterisation import normalise_illuminant, training_data_sds_to_RGB
import numpy as np
MSDS_TRAIN = load_camera(CAMERA)

response_trainset_camera = msds_to_rgb(TRAIN,MSDS_TRAIN, illuminant) * 2
response_testset_camera = msds_to_rgb(TEST,MSDS_TRAIN, illuminant)
response_validationset_camera = msds_to_rgb(VALIDATION,MSDS_TRAIN, illuminant)


[  938.7258382   1714.98576086  1939.82119587]
[  938.7258382   1714.98576086  1939.82119587]
[  938.7258382   1714.98576086  1939.82119587]
[]


### Fit Generalized Additive Model with P-splines

### Nikon

In [41]:
from sklearn.metrics import make_scorer
from colour_math import deltae_mean
from models import GAMOptimizer
from evaluate import pred
np.int = np.int64
if CAMERA == 'nikon':
    # gam = GAMOptimizer(lams=1e-6, order=3, n_splines=5)
    
    # gam = GAMOptimizer(lams=0.0001,order=3, n_splines=10)
    gam = GAMOptimizer(lams=0.01,order=3, n_splines=20)

    gam.fit(response_trainset_camera, response_trainset_xyz)

    pred(gam, response_testset_camera, response_testset_xyz, "DeltaE Foster+CAVE")

### Sigma

In [45]:
from sklearn.metrics import make_scorer
from colour_math import deltae_mean
from models import GAMOptimizer
from evaluate import pred
np.int = np.int64
if CAMERA == 'sigma':
    # gam = GAMOptimizer(lams=1e-9, order=3, n_splines=5)
    # gam = GAMOptimizer(lams=0.0001,order=3, n_splines=10)
    gam = GAMOptimizer(lams=0.0001,order=3, n_splines=20)

    gam.fit(response_trainset_camera, response_trainset_xyz)

    pred(gam, response_testset_camera, response_testset_xyz, "DeltaE Foster+CAVE")

3600
---- RESULTS DeltaE Foster+CAVE ----
DeltaE mean: 1.73
DeltaE max: 16.95
DeltaE median: 1.46
DeltaE 95 percentile: 4.00
DeltaE 99 percentile: 5.94


### Fit Linear Model

In [43]:
from sklearn.linear_model import LinearRegression

linear = LinearRegression(fit_intercept=False)


linear.fit(response_trainset_camera, response_trainset_xyz)
pred(linear, response_testset_camera, response_testset_xyz, "Foster 50")


---- RESULTS Foster 50 ----
DeltaE mean: 2.20
DeltaE max: 23.49
DeltaE median: 1.52
DeltaE 95 percentile: 6.98
DeltaE 99 percentile: 10.56


In [46]:
from sklearn.pipeline import Pipeline
from models import DeltaEOptimizer

DE2000RP = Pipeline([
    ('regressor', DeltaEOptimizer(root_polynomial=False, degree=1))
])


DE2000RP.fit(response_trainset_camera, response_trainset_xyz)
pred(DE2000RP, response_testset_camera, response_testset_xyz, "DeltaE Foster+CAVE")

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 2.6196741857032744
        x: [ 8.371e-01 -5.934e-01  6.989e-01 -4.839e-01  1.788e+00
            -2.987e-01  9.454e-01 -4.451e+00  4.568e+00]
      nit: 30
      jac: [-9.537e-07 -3.874e-07 -5.960e-08  2.295e-06  1.490e-06
             1.073e-06 -1.162e-06 -7.749e-07 -6.855e-07]
 hess_inv: [[ 1.262e+00 -2.825e+00 ..., -2.501e+00  1.544e+00]
            [-2.825e+00  7.011e+00 ...,  6.297e+00 -4.036e+00]
            ..., 
            [-2.501e+00  6.297e+00 ...,  1.045e+01 -6.912e+00]
            [ 1.544e+00 -4.036e+00 ..., -6.912e+00  4.737e+00]]
     nfev: 370
     njev: 37
---- RESULTS DeltaE Foster+CAVE ----
DeltaE mean: 2.57
DeltaE max: 19.60
DeltaE median: 2.15
DeltaE 95 percentile: 5.59
DeltaE 99 percentile: 7.84


### Fit 3rd order Root-Polynomial Model

In [47]:
from models import PolynomialTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

RP_linear_3 = Pipeline([
    ('transformer', PolynomialTransformer(degree=3, rp=True)),
    ('regressor', LinearRegression(fit_intercept=False))
])

RP_linear_3.fit(response_trainset_camera, response_trainset_xyz)

pred(RP_linear_3, response_testset_camera, response_testset_xyz, "DeltaE Foster+CAVE")


---- RESULTS DeltaE Foster+CAVE ----
DeltaE mean: 2.32
DeltaE max: 34.32
DeltaE median: 1.65
DeltaE 95 percentile: 6.90
DeltaE 99 percentile: 11.37


### Fit a 2nd order Root-Polynomial Model

In [48]:
from models import GAMOptimizer, PolynomialTransformer, DeltaEOptimizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

RP_linear_2 = Pipeline([
    ('transformer', PolynomialTransformer(degree=2, rp=True)),
    ('regressor', LinearRegression(fit_intercept=False))
])


RP_linear_2.fit(response_trainset_camera, response_trainset_xyz)
pred(RP_linear_2, response_testset_camera, response_testset_xyz, "DeltaE Foster+CAVE")


---- RESULTS DeltaE Foster+CAVE ----
DeltaE mean: 2.17
DeltaE max: 25.38
DeltaE median: 1.50
DeltaE 95 percentile: 6.84
DeltaE 99 percentile: 10.04


In [53]:
P_Linear_3 = Pipeline([
    ('transformer', PolynomialTransformer(degree=3, rp=False)),
    ('regressor', LinearRegression(fit_intercept=False))
])


P_Linear_3.fit(response_trainset_camera, response_trainset_xyz)
pred(P_Linear_3, response_testset_camera, response_testset_xyz, "DeltaE Foster+CAVE")

---- RESULTS DeltaE Foster+CAVE ----
DeltaE mean: 1.92
DeltaE max: 20.78
DeltaE median: 1.49
DeltaE 95 percentile: 5.01
DeltaE 99 percentile: 8.16


In [52]:
P_linear = Pipeline([
    ('transformer', PolynomialTransformer(degree=2, rp=False)),
    ('regressor', LinearRegression(fit_intercept=False))
])


P_linear.fit(response_trainset_camera, response_trainset_xyz)
pred(P_linear, response_testset_camera, response_testset_xyz, "DeltaE Foster+CAVE")


---- RESULTS DeltaE Foster+CAVE ----
DeltaE mean: 2.02
DeltaE max: 22.07
DeltaE median: 1.49
DeltaE 95 percentile: 5.82
DeltaE 99 percentile: 9.20


In [54]:
from sklearn.pipeline import Pipeline
from models import DeltaEOptimizer

DE2000P = Pipeline([
    ('regressor', DeltaEOptimizer(root_polynomial=False, degree=3))
])


DE2000P.fit(response_trainset_camera, response_trainset_xyz)
pred(DE2000P, response_testset_camera, response_testset_xyz, "DeltaE Foster+CAVE")

  message: Desired error not necessarily achieved due to precision loss.
  success: False
   status: 2
      fun: 1.5397398602444612
        x: [ 8.418e-01 -6.156e-01 ..., -7.223e+00  2.446e+00]
      nit: 244
      jac: [-7.191e-04 -5.947e-04 ...,  9.537e-05  6.939e-05]
 hess_inv: [[ 2.272e+00 -4.223e+00 ..., -1.291e+01 -9.161e-01]
            [-4.223e+00  9.763e+00 ...,  9.804e+00  1.391e+01]
            ..., 
            [-1.291e+01  9.804e+00 ...,  2.793e+03 -1.602e+03]
            [-9.161e-01  1.391e+01 ..., -1.602e+03  1.347e+03]]
     nfev: 18917
     njev: 326
---- RESULTS DeltaE Foster+CAVE ----
DeltaE mean: 1.98
DeltaE max: 17.92
DeltaE median: 1.74
DeltaE 95 percentile: 4.42
DeltaE 99 percentile: 6.37


In [55]:
from sklearn.pipeline import Pipeline
from models import DeltaEOptimizer

DE2000RP = Pipeline([
    ('regressor', DeltaEOptimizer(root_polynomial=True, degree=3))
])


DE2000RP.fit(response_trainset_camera, response_trainset_xyz)
pred(DE2000RP, response_testset_camera, response_testset_xyz, "DeltaE Foster+CAVE")

  message: Desired error not necessarily achieved due to precision loss.
  success: False
   status: 2
      fun: 1.5985979848044964
        x: [ 5.327e+01 -4.380e+02 ..., -3.162e+01 -8.182e+01]
      nit: 886
      jac: [-3.651e-06 -7.506e-05 ...,  7.200e-05  7.273e-05]
 hess_inv: [[ 2.521e+04  3.125e+04 ...,  2.189e+04  3.597e+04]
            [ 3.125e+04  4.436e+04 ...,  2.572e+04  3.509e+04]
            ..., 
            [ 2.189e+04  2.572e+04 ...,  2.736e+04  3.628e+04]
            [ 3.597e+04  3.509e+04 ...,  3.628e+04  2.667e+05]]
     nfev: 42800
     njev: 1070
---- RESULTS DeltaE Foster+CAVE ----
DeltaE mean: 2.00
DeltaE max: 43.94
DeltaE median: 1.35
DeltaE 95 percentile: 5.51
DeltaE 99 percentile: 9.70
